In [1]:
import os
import sys
#sys.path.append("/workspace")
import numpy as np
import tensorflow as tf
from src.data.general_processor import Utils
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle
from sklearn.preprocessing import minmax_scale
import pathlib
from sklearn.metrics import classification_report, confusion_matrix


from src.models.model_architectures.model_1DCNN import HopefullNet

## Get test data 4+1 classes

In [8]:
root_path = pathlib.Path().resolve().parents[0]

data_path = "data/processed/physionet/4+1_classes/paper_channel_pair_d/"
modelname = "original_roi_d/"

## Device settings
tf.autograph.set_verbosity(0)
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.experimental.list_physical_devices('CPU')
print(physical_devices)
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

#Params
source_path = os.path.join(root_path, data_path)
save_path = os.path.join(root_path, "models/1D_CNN/", modelname)

try:
    os.mkdir(save_path)
except OSError as error:
    print(error)  

# Load data
channels = Utils.combinations["d"] #[["C5", "C6"], ["C3", "C4"], ["C1", "C2"]]

exclude =  [38, 88, 89, 92, 100, 104]
subjects = [n for n in np.arange(1,110) if n not in exclude]

#Load data
x, y = Utils.load(channels, subjects, base_path=source_path)

#Transform y to one-hot-encoding
y_one_hot  = Utils.to_one_hot(y, by_sub=False)

#Reshape for scaling
reshaped_x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])

#Grab a test set before SMOTE
x_train_raw, x_valid_test_raw, y_train_raw, y_valid_test_raw = train_test_split(reshaped_x,
                                                                            y_one_hot,
                                                                            stratify=y_one_hot,
                                                                            test_size=0.20,
                                                                            random_state=42)

#Scale indipendently train/test
#x_train_scaled_raw = minmax_scale(x_train_raw, axis=1)
x_test_valid_scaled_raw = minmax_scale(x_valid_test_raw, axis=1)

#Create Validation/test
x_valid_raw, x_test_raw, y_valid, y_test = train_test_split(x_test_valid_scaled_raw,
                                                    y_valid_test_raw,
                                                    stratify=y_valid_test_raw,
                                                    test_size=0.50,
                                                    random_state=42)

#x_valid = x_valid_raw.reshape(x_valid_raw.shape[0], int(x_valid_raw.shape[1]/2),2).astype(np.float64)
x_test = x_test_raw.reshape(x_test_raw.shape[0], int(x_test_raw.shape[1]/2),2).astype(np.float64)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\annag\\OneDrive - Danmarks Tekniske Universitet\\Semester_04\\Special_Course_BCI\\03_code\\BCI_stroke_rehab\\models/1D_CNN/original_roi_d/'


## Get test data 2 classes

In [3]:
root_path = pathlib.Path().resolve().parents[0]
data_path = "data/processed/physionet/2_classes/ours_3_pairs/"
modelname = "ours_3_pairs/2023-03-31_00-13-36/"

## Device settings
tf.autograph.set_verbosity(0)
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.experimental.list_physical_devices('CPU')
print(physical_devices)
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

#Params
source_path = os.path.join(root_path, data_path)
save_path = os.path.join(root_path, "models/1D_CNN/", modelname)

try:
    os.mkdir(save_path)
except OSError as error:
    print(error)  

# Load data
channels = [["C3","C4"],["F3","F4"],["P3","P4"]]

exclude =  [38, 88, 89, 92, 100, 104]
subjects = [n for n in np.arange(1,110) if n not in exclude]

#Load data
x, y = Utils.load(channels, subjects, base_path=source_path)

# #Transform y to one-hot-encoding
# y_one_hot  = Utils.to_one_hot(y, by_sub=False)
y_one_hot = y

#Reshape for scaling
reshaped_x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])

#Grab a test set before SMOTE
x_train_raw, x_valid_test_raw, y_train_raw, y_valid_test_raw = train_test_split(reshaped_x,
                                                                            y_one_hot,
                                                                            stratify=y_one_hot,
                                                                            test_size=0.20,
                                                                            random_state=42)
#Scale indipendently train/test
#x_train_scaled_raw = minmax_scale(x_train_raw, axis=1)
x_test_valid_scaled_raw = minmax_scale(x_valid_test_raw, axis=1)

#Create Validation/test
x_valid_raw, x_test_raw, y_valid, y_test = train_test_split(x_test_valid_scaled_raw,
                                                    y_valid_test_raw,
                                                    stratify=y_valid_test_raw,
                                                    test_size=0.50,
                                                    random_state=42)

#x_valid = x_valid_raw.reshape(x_valid_raw.shape[0], int(x_valid_raw.shape[1]/2),2).astype(np.float64)
x_test = x_test_raw.reshape(x_test_raw.shape[0], int(x_test_raw.shape[1]/2),2).astype(np.float64)

y_test_01 = []
for y_label in y_test:
    if y_label == 'L':
        y_test_01.append(0)
    elif y_label == 'R':
        y_test_01.append(1)
    else:
        print("Test Labels are different than L or R...")

y_test = np.array(y_test_01)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\annag\\OneDrive - Danmarks Tekniske Universitet\\Semester_04\\Special_Course_BCI\\03_code\\BCI_stroke_rehab\\models/1D_CNN/ours_3_pairs/2023-03-31_00-13-36/'


## Get model

In [6]:
model_path = "C:/Users/annag/OneDrive - Danmarks Tekniske Universitet/Semester_04/Special_Course_BCI/03_code/BCI_stroke_rehab/models/1D_CNN/ours_3_pairs/2023-03-31_00-13-36/bestModel.h5"
two_class = True

if two_class:
    input_shape = (None, 500, 2)
    loss = tf.keras.losses.binary_crossentropy
else:
    input_shape = (None, 640, 2)
    loss = tf.keras.losses.categorical_crossentropy

In [7]:
model = HopefullNet(inp_shape = (input_shape[1],input_shape[2]), two_class=two_class)
model.build(input_shape)
model.load_weights(model_path)

learning_rate = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

Input shape: (500, 2)
Predicting 2 classes.
Input shape: (None, 500, 2)
Output shape: (None, 1)


In [9]:
testLoss, testAcc = model.evaluate(x_test, y_test)
print('\nAccuracy:', testAcc)
print('\nLoss: ', testLoss)

yPred = model.predict(x_test)

# convert from one hot encode in string
if two_class:
    yTestClass = []
    for label in y_test:
        if label==0:
            yTestClass.append("L")
        elif label==1:
            yTestClass.append("R")
        else:
            print("Label not found.")
               
    yPredClass = []
    for label in yPred:
        if label<0.5:
            yPredClass.append("L")
        elif label>=0.5:
            yPredClass.append("R")
        else:
            print("Label not found.")
    
    target_names = ["L", "R"]
else:  
    yTestClass = np.argmax(y_test, axis=1)
    yPredClass = np.argmax(yPred,axis=1)
    target_names=["B", "R", "RL", "L", "F"]

print('\n Classification report \n\n',
  classification_report(
      yTestClass,
      yPredClass,
       target_names=target_names
  )
)

print('\n Confusion matrix \n\n',
  confusion_matrix(
      yTestClass,
      yPredClass,
  )
)

44/44 [==============================] - 1s 21ms/step - loss: 0.3531 - accuracy: 0.8661

Accuracy: 0.8661360144615173

Loss:  0.3530680239200592
44/44 [==============================] - 1s 21ms/step

 Classification report 

               precision    recall  f1-score   support

           L       0.87      0.87      0.87       698
           R       0.86      0.87      0.87       684

    accuracy                           0.87      1382
   macro avg       0.87      0.87      0.87      1382
weighted avg       0.87      0.87      0.87      1382


 Confusion matrix 

 [[604  94]
 [ 91 593]]
